# OpenStudio Refrigeration System Modeling - Main Execution Notebook

This notebook guides you through the full pipeline for generating refrigeration system JSON files using either:
- **SuperMarket mode** (automated)
- **User-defined mode** (manual input)

## Step 1: Imports & Setup

In [1]:
import sys
sys.path.append(".")
from refrigeration.mode_selection import (
    automated_mode,
    user_mode,
    get_valid_template,
    select_test_mode
)
from refrigeration.rack_assignment import (
    assign_racks_to_cases_and_walkins,
    display_rack_capacity
)
from refrigeration.compressor import (
    summarize_compressor_assignment,
    prepare_and_store_compressor_objects
)
from refrigeration.condenser import prepare_and_store_condenser_objects
from refrigeration.case_walkin_objects import (
    generate_case_objects_from_data,
    generate_walkin_objects_from_data,
    prepare_and_store_case_and_walkin_objects
)
from refrigeration.system_objects import (
    prepare_and_store_system_and_casewalkin_lists,
    generate_system_and_casewalkin_lists
)
from refrigeration.json_io import (
    export_existing_compressors_to_json,
    export_existing_condensers_to_json,
    export_cases_and_walkins_to_json,
    export_system_and_casewalkin_lists_to_json
)
from refrigeration.full_export import export_full_refrigeration_system_to_json
from refrigeration.utils import get_building_name

# set DB path 
db_path = "database/openstudio_refrigeration_system.db"

## Step 2: Select Mode (User / Automated)

In [2]:
mode = select_test_mode()

if mode == "user":
    selected_case_units, selected_walkin_units, selected_template = user_mode()
elif mode == "automated":
    selected_case_units, selected_walkin_units, selected_template = automated_mode(db_path)

# Selected Case Units
print("\nSelected Case Units:")
for unit in selected_case_units:
    print(f"\"osm name\": \"{unit.osm_name}\", \"case_name\": \"{unit.case_name}\", \"number_of_units\": {unit.number_of_units}")

# Selected Walk-in Units
print("\nSelected Walk-in Units:")
for unit in selected_walkin_units:
    print(f"\"osm name\": \"{unit.osm_name}\", \"walkin_name\": \"{unit.walkin_name}\", \"number_of_units\": {unit.number_of_units}")

Select test mode (user/automated): automated
Choose building type:
1. SuperMarket
2. TBD
Enter the number of your choice: 1
Chosen building type: SuperMarket
Choose template (old/new/advanced): advanced

Selected Case Units:
"osm name": "SuperMarket advanced LT Coffin - Ice Cream - Ice Cream", "case_name": "advanced LT Coffin - Ice Cream", "number_of_units": 1.0
"osm name": "SuperMarket advanced LT Reach-In - Ice Cream - Ice Cream", "case_name": "advanced LT Reach-In - Ice Cream", "number_of_units": 14.0
"osm name": "SuperMarket advanced LT Reach-In - Frozen Food - Frozen Food", "case_name": "advanced LT Reach-In - Frozen Food", "number_of_units": 14.0
"osm name": "SuperMarket advanced MT Island - Deli Produce Case - Deli", "case_name": "advanced MT Island - Deli Produce Case", "number_of_units": 14.0
"osm name": "SuperMarket advanced MT Island - Deli Produce Case - Produce", "case_name": "advanced MT Island - Deli Produce Case", "number_of_units": 10.0
"osm name": "SuperMarket advance

## Step 3: Rack Assignment

In [4]:
# Assign refrigeration units to racks and retrieve updated data
mt_racks, lt_racks, case_data, walkin_data = assign_racks_to_cases_and_walkins(
    db_path, selected_case_units, selected_walkin_units
)

# display capacity distribution for assigned racks
display_rack_capacity(mt_racks, selected_case_units, rack_type="MT")
display_rack_capacity(lt_racks, selected_walkin_units, rack_type="LT")


MT Racks:
Rack 1: Total Capacity = 81644.64 W
  - SuperMarket advanced MT Vertical Open - Others - Prepared Foods : 81644.64 W

Rack 2: Total Capacity = 25132.32 W
  - SuperMarket advanced MT Vertical Open - Meat - Meat : 25132.32 W

Rack 3: Total Capacity = 29896.18 W
  - SuperMarket advanced MT Service - Others - Bakery : 10385.28 W
  - SuperMarket advanced MT Vertical Open - Beverage - Beverage : 7039.20 W
  - advanced MT Walk-in Cooler - 660SF with glass door : 6460.70 W
  - advanced MT Walk-in Cooler - 600SF with no glass door : 6011.00 W

Rack 4: Total Capacity = 25574.20 W
  - SuperMarket advanced MT Service - Meat - Meat : 5504.40 W
  - advanced MT Walk-in Cooler - 480SF with no glass door : 5085.30 W
  - advanced MT Walk-in Cooler - 480SF with glass door : 5085.30 W
  - advanced MT Walk-in Cooler - 400SF with no glass door : 4448.80 W
  - advanced MT Walk-in Cooler - 120SF with no glass door : 2098.20 W
  - advanced MT Walk-in Cooler - 80SF with no glass door : 1746.90 W
  - 

# Step 4: Compressors + Condensers

In [5]:
#calculate compfessor assignemnt for MT and LT racks
mt_info, lt_info = summarize_compressor_assignment(mt_racks, lt_racks, selected_template)

#Generate and store compressor and condenser object
compressor_data = prepare_and_store_compressor_objects(mt_info, lt_info, selected_template, db_path)
condenser_data = prepare_and_store_condenser_objects(mt_info, lt_info, selected_template)


🧊 MT Rack Compressor Assignment:
Rack 1: Load = 81644.64 W → Compressors Needed = 4
Rack 2: Load = 25132.32 W → Compressors Needed = 2
Rack 3: Load = 29896.18 W → Compressors Needed = 2
Rack 4: Load = 25574.20 W → Compressors Needed = 2

❄️ LT Rack Compressor Assignment:
Rack 1: Load = 17623.20 W → Compressors Needed = 3
Rack 2: Load = 16753.80 W → Compressors Needed = 2
Rack 3: Load = 663.60 W → Compressors Needed = 2

⚙️ Compressor Specs for the selected template 'advanced':
🧊 MT → Capacity: 38099.93 W, Power: 15448.00 W, COP: 2.47, EER: 8.42
❄️ LT → Capacity: 17181.96 W, Power: 9766.00 W, COP: 1.76, EER: 6.00
✅ Compressor objects and performance curves ready.
✅ Condenser and curve objects generated and stored in globals.


# Step 5: Generate and Store Case and Walk-in objects

In [6]:
prepare_and_store_case_and_walkin_objects(case_data, walkin_data, selected_case_units, selected_walkin_units)

✅ Case and walk-in objects generated and stored in globals.


# Step 6: Generate and Store Refrigeration Systems and Case/Walkin Lists

In [7]:
# Generate and store System and Case/Walkin List objects
system_objects = generate_system_and_casewalkin_lists(
    selected_case_units,
    selected_walkin_units,
    mt_racks,
    lt_racks,
    selected_template
)

# Pass the result to the next function instead of relying on globals
prepare_and_store_system_and_casewalkin_lists(
    selected_case_units,
    selected_walkin_units,
    mt_racks,
    lt_racks,
    selected_template
)

✅ Refrigeration system objects stored in globals.


# Step 7: Preivew and Export JSON Files

In [11]:
# Prepare and store compressor objects
mt_compressors, lt_compressors, mt_power_curve, mt_capacity_curve, lt_power_curve, lt_capacity_curve = prepare_and_store_compressor_objects(
    mt_info, lt_info, selected_template, db_path
)

# Export compressor data to JSON
export_existing_compressors_to_json(
    mt_compressors=mt_compressors,
    lt_compressors=lt_compressors,
    mt_power_curve=mt_power_curve,
    mt_capacity_curve=mt_capacity_curve,
    lt_power_curve=lt_power_curve,
    lt_capacity_curve=lt_capacity_curve,
    output_path="All_Compressors.json"
)


✅ Compressor objects and performance curves ready.
✅ Compressor + Curve JSON with zones saved to: All_Compressors.json

📦 OpenStudio JSON Preview:

{
  "Version": "0.2.1",
  "Building": "User Defined System",
  "objects": [
    {
      "type": "OS:ThermalZone",
      "name": "MainSales"
    },
    {
      "type": "OS:ThermalZone",
      "name": "ActiveStorage"
    },
    "mt_power_curve",
    "mt_capacity_curve",
    "lt_power_curve",
    "lt_capacity_curve",
    "mt_compressors",
    "lt_compressors"
  ]
}


In [9]:
print(type(mt_power_curve))
print(type(mt_capacity_curve))
print(type(lt_power_curve))
print(type(lt_capacity_curve))
print(type(mt_compressors))
print(type(lt_compressors))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [ ]:
prepare_and_store_compressor_objects(mt_info, lt_info, selected_template, db_path)
export_existing_condensers_to_json_from_globals("All_Condensers.json")


In [ ]:
export_cases_and_walkins_to_json_from_globals("All_Cases_Walkins.json")


In [ ]:
export_refrigeration_system_objects_to_json_from_global("CaseWalkin_List.json")

In [ ]:

export_full_refrigeration_system_to_json_from_globals("Full_Refrigeration_System.json")
print("🏁 Refrigeration JSON export complete.")
print(f"Building: {get_building_name()}")